# Lecture 3a: EEG Signal Analysis for Interactive Systems
Signal Processing for Interactive Systems

Cumhur Erkut (cer@create.aau.dk)

Aalborg University Copenhagen

Last edited: 2025-03-02

# EEG Time-Frequency Analysis for Interactive Applications
This notebook demonstrates key spectral and time-frequency methods for EEG analysis.
It covers:
- **Time-domain representation**
    - **Autocorrelation** (Identifying periodicity in EEG signals)
    - **Autoregressive (AR) Model** (Parametric spectral estimation)
- **Frequency-domain techniques**
    - **Periodogram** (Power Spectral Density estimation)
    - **Welch's method** (Improved PSD estimation using averaging)
    - **Multitaper Method** (Variance reduction in spectral estimation)
- **Time-frequency methods**
    - **Short-Time Fourier Transform (STFT)** (Time-frequency analysis)
- **Other methods, recent developments** 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import periodogram, welch, correlate, stft, spectrogram, windows
from spectrum import pmtm, aryule, marple_data

## 1. Introduction
EEG signals are oscillatory signals that are commonly analyzed in both the time and frequency domains. 
The fundamental goal of spectral estimation is to determine how power is distributed across different frequencies.
 
Here, the EEG signal is modeled as: 
$$ x(t) = A_1 \sin(2\pi f_1 t) + A_2 \sin(2\pi f_2 t) + n(t) $$
where *f*~1~, and *f*~2~ are the frequencies present in the signal, and *n(t)* is noise.


In [ ]:
# Simulated EEG Signal Parameters
fs = 256  # Sampling frequency (Hz)
t = np.arange(0, 10, 1/fs)  # 10 seconds of data
signal = np.sin(2 * np.pi * 10 * t) + np.sin(2 * np.pi * 20 * t) + np.random.normal(0, 0.5, len(t))


## 2. Time-Domain Representation


In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(t[:1000], signal[:1000])  # Show first few seconds
plt.title("Simulated EEG Signal in Time Domain")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.show()

### Autocorrelation Analysis
Autocorrelation identifies periodicities in the signal by computing:
$$ R_x(\tau) = \sum_{n} x[n] x[n - \tau] $$


In [ ]:
corr = correlate(signal, signal, mode='full')
lag = np.arange(-len(signal) + 1, len(signal)) / fs
plt.figure()
plt.plot(lag, corr)
plt.title("Autocorrelation of EEG Signal")
plt.xlabel("Lag (s)")
plt.ylabel("Correlation")
plt.show()


### Autoregressive (AR) Model
The AR model estimates the power spectrum by fitting the data to a parametric model:

$$ x[n] = \sum_{p=1}^{P} a_p x[n - p] + e[n] $$


In [ ]:
ar_order = 10
ar_coeffs, noise_var = aryule(signal, ar_order)
w, h = np.linspace(0, fs/2, len(signal)), np.abs(1 / np.polyval(np.concatenate(([1], -ar_coeffs)), np.exp(-2j * np.pi * np.linspace(0, 0.5, len(signal)))))
plt.figure()
plt.semilogy(w * fs, h**2 * noise_var)
plt.title("Autoregressive Model Power Spectral Density")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.show()

## 3. Spectral Estimation
### 3.1 Periodogram Method
The periodogram estimates the power spectral density (PSD) using the squared magnitude of the Fourier Transform:
$$ P(f) = \frac{1}{N} \left| X(f) \right|^2 $$
where *X(f)* is the Discrete Fourier Transform (DFT) of the signal.


In [ ]:
freqs, psd = periodogram(signal, fs=fs)
plt.figure()
plt.semilogy(freqs, psd)
plt.title("Periodogram Power Spectral Density")
plt.xlabel("Frequency (Hz)")
plt.xlim(0.5, 100)
plt.ylim(0.000001, 10)
plt.xticks(np.arange(10, 100, 10))
plt.ylabel("Power")
plt.show()

### 3.2 Welch's Method**
Welch’s method improves PSD estimation by dividing the signal into overlapping segments and averaging the periodograms:
$$ P_{Welch}(f) = \frac{1}{K} \sum_{k=1}^{K} P_k(f) $$

In [ ]:
freqs_welch, psd_welch = welch(signal, fs=fs, nperseg=1024)
plt.figure()
plt.semilogy(freqs_welch, psd_welch)
plt.title("Welch's Method Power Spectral Density")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.show()

### 3.3 Multitaper Method**
The multitaper method reduces spectral variance by applying multiple tapers to the signal:
$$ P_{MT}(f) = \frac{1}{K} \sum_{k=1}^{K} \left| X_k(f) \right|^2 $$


In [ ]:
Sk, weights, _ = pmtm(signal, NW=3.5, show=False)
freqs_mt = np.linspace(0, fs/2, len(Sk))
plt.figure()
plt.semilogy(freqs_mt, np.mean(Sk, axis=1))
plt.title("Multitaper Method Power Spectral Density")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.show()

## 4. Time-Frequency Analysis
###  4.1 Short-Time Fourier Transform (STFT)
STFT applies the Fourier transform over short time windows:
$$ STFT \{ x(t) \} (\tau, f) = \int x(t) w(t - \tau) e^{-j2\pi ft} dt $$



In [ ]:
f, t, Zxx = stft(signal, fs, nperseg=256)
plt.figure()
plt.pcolormesh(t, f, np.abs(Zxx), shading='gouraud')
plt.title("Short-Time Fourier Transform (STFT)")
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")
plt.colorbar(label="Magnitude")
plt.show()

# Summary
In time-domian,
- **Autocorrelation** helps reveal periodic components.
- **Autoregressive Model** provides a smooth parametric spectral estimation (gateway to frequency domain).
In frequency-domain
- **Periodogram** provides a simple power spectrum estimate but is noisy.
- **Welch's method** improves stability by averaging over overlapping segments.
- **Multitaper method** reduces variance for a more stable spectrum.
In time-frequency,
- **STFT** enables time-frequency visualization of EEG signals.
- Is a gateway to other time-frequency tilings.

## Activities

- Study other methods from the paper: Zhang-2019-EEGTF
- Run all techniques on the real data 
- Run all techniques within MNE, on [simulated](https://mne.tools/stable/auto_examples/time_frequency/time_frequency_simulated.html) or [real data](https://mne.tools/stable/auto_tutorials/intro/10_overview.html#time-frequency-analysis) 
- Check how to apply [EEG data to Machine Learning on Edge Impulse](https://docs.edgeimpulse.com/experts/novel-sensor-projects/eeg-data-machine-learning-part-1)

In [ ]:
# Starter code for real data (MATLAB format)
try:
  import google.colab
  IN_COLAB = True
  !mkdir -p data
  !wget https://raw.githubusercontent.com/SMC-AAU-CPH/SPIS/main/03-Fourier-Transfom/data/NewEEGSignal.mat -P data
except:
  IN_COLAB = False

from scipy.io import loadmat

# Load the signal from the .mat file
signal = loadmat('data/NewEEGSignal.mat')['NewEEGSignal'][0]
fs = 256 #Hz
t = np.arange(0, 512) / fs
plt.figure(figsize=(10, 4))
plt.plot(t[:1000], signal[:1000])  # Show first few seconds
plt.title("Real EEG Signal in Time Domain")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.show()

# etc